# Encoder Estimation

## Author: Maneeshika Madduri

### Goal
This code takes EMG data and estimtes an encoding model such that:

EMG = Encoder $\times$ task data

From the EMG data, we estimate an encoder

### Encoder Model

$$ EMG = W*u $$

$$ u = 
\begin{bmatrix}
r_x \\ r_y \\ \dot{r_x} \\ \dot{r_y}  \\ r_x - p_x \\ r_y - p_y \\ \dot{r_x} - \dot{p_x}   \\ \dot{r_y} - \dot{p_y} \\ 1
\end{bmatrix}
$$

where 
$$
FF = \begin{bmatrix}
r_x \\ r_y \\ \dot{r_x} \\ \dot{r_y} 
\end{bmatrix}
,
FB = \begin{bmatrix}
r_x - p_x \\ r_y - p_y \\ \dot{r_x} - v_{dec,x}  \\ \dot{r_y} - v_{dec,y} 
\end{bmatrix}
,
b = \begin{bmatrix}
 1
\end{bmatrix}
$$

So $W \in R^{64 \times 9}$

### STEP 1: Load Python Packages

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import wilcoxon as wilcoxon

# meta analysis functions
import sys
sys.path.append('/code/')
from util import analysis
from util import plotting
from util import util_continuous as utils

# seaborn.set()

### STEP 2: Load Data

In [ ]:
# path = '/Volumes/My Passport/cphs'
path = '/data/'

with open(path + '/continuous_full_data_block1_sorted.pickle', 'rb') as handle:
    refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1, Hs_block1, alphas_block1, pDs_block1, times_block1, conditions_block1 = pickle.load(handle)
keys = ['METACPHS_S106', 'METACPHS_S107','METACPHS_S108', 'METACPHS_S109', 'METACPHS_S110', 'METACPHS_S111', 'METACPHS_S112', 'METACPHS_S113', 'METACPHS_S114', 'METACPHS_S115', 'METACPHS_S116', 'METACPHS_S117', 'METACPHS_S118', 'METACPHS_S119']

with open(path + '/continuous_full_data_block2_sorted.pickle', 'rb') as handle:
    refs_block2, poss_block2, dec_vels_block2, int_vel_block2, emgs_block2, Ws_block2, Hs_block2, alphas_block2, pDs_block2, times_block2, conditions_block2 = pickle.load(handle)
keys = ['METACPHS_S106', 'METACPHS_S107','METACPHS_S108', 'METACPHS_S109', 'METACPHS_S110', 'METACPHS_S111', 'METACPHS_S112', 'METACPHS_S113', 'METACPHS_S114', 'METACPHS_S115', 'METACPHS_S116', 'METACPHS_S117', 'METACPHS_S118', 'METACPHS_S119']

# reference (aka target) and reference velocity (aka target velocity) data 
with open(path + '/ref_vel_data.pickle', 'rb') as handle:
    times, refs, ref_vels = pickle.load(handle)

### STEP 3: Set up utils

Experimental Utils

saved as:

0 = D_1 (fast, pd3 = high $\lambda$, +)

1 = D_2 (fast, pd4 = low $\lambda$, +)

2 = D_5 (fast, pd3 = high $\lambda$, -)

3 = D_6 (fast, pd4 = low $\lambda$, -)

4 = D_3 (slow, pd3 = high $\lambda$, +)

5 = D_4 (slow, pd4 = low $\lambda$, +)

6 = D_7 (slow, pd3 = high $\lambda$, -)

7 = D_8 (slow, pd4 = low $\lambda$, -)

In [ ]:
key = keys[0]
alphas = alphas_block1[key]
print(alphas)

conds = conditions_block1[key]
print(conds)

pDs = pDs_block1[key]
print(pDs)

# initialization conditions
pos_init = [0, 1, 4, 5]
neg_init = [2, 3, 6, 7]

# penalty parameter conditions
pD_3 = [0, 2, 4, 6] # pD = 1e-3
pD_4 = [1, 3, 5, 7] # pD = 1e-4

# learning rate conditions
slow = [4, 5, 6, 7] 
fast = [0, 1, 2, 3]

# dimensions
n_keys = len(keys) # number of subjects = length of keys
n_blocks = 2 # number of blocks = 2
# number of decoder conditions, time samples, number of emg channels 
n_conds, n_time, n_feat = emgs_block1[keys[0]].shape 
n_dim = 2 # number of dimensions of the task (2D trajectory tracking)

# set the time of the ramp at the start of each trial
time_x = times_block1[keys[0]][0]
ramp = np.where(time_x > 5)[0][0]
print(ramp)

Code to check that separation has been run correctly

Note: Checked Oct 12 2023

In [ ]:
# check that the separation is correct 


for key in keys:
   assert(np.all(alphas_block1[key][slow] == 0.75))
   assert(np.all(alphas_block2[key][slow] == 0.75))
   assert(np.all(alphas_block1[key][fast] == 0.25))
   assert(np.all(alphas_block2[key][fast] == 0.25))

for key in keys:
    # check all elements of positive init decoder are greater than 0
    assert((Ws_block1[key][pos_init][:, 0] > 0).all() == True)
    assert((Ws_block2[key][pos_init][:, 0] > 0).all() == True)

    # check all elements of negative init decoder are greater than 0
    assert((Ws_block1[key][neg_init][:, 0] < 0).all() == True)
    assert((Ws_block2[key][neg_init][:, 0] < 0).all() == True)

for key in keys:
    # check all penalty paramters are sorted correctly
    assert((pDs_block1[key][pD_3] == 1e-3).all())
    assert((pDs_block2[key][pD_3] == 1e-3).all())

    # check all penalty paramters are sorted correctly
    assert((pDs_block1[key][pD_4] == 1e-4).all())
    assert((pDs_block2[key][pD_4] == 1e-4).all())

    

    


Plotting Utils

In [ ]:
from matplotlib import rcParams


BLUE = '#1f77b4',
ORANGE = '#ff7f0e',
GREEN = '#2ca02c',
RED = '#d62728',
PURPLE = '#9467bd',
GOLD = '#FFD700'

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#FFD700', '#BDBDBD', '#676767']

## set up plots
rcParams['font.weight'] = 'ultralight'
rcParams['font.family'] = 'sans serif'
rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.size'] = 15

rcParams['text.color'] = 'black'
rcParams['axes.labelcolor'] = 'black'
rcParams['xtick.color'] = 'black'
rcParams['ytick.color'] = 'black'

rcParams["legend.frameon"] = False

In [ ]:
# trying to figure out how often decoder updates -- every 1202 samples

W = Ws_block1[keys[0]][0]
W[1:,:,:].shape # 7199 time points x (decoder dimensions is 2 x 6)
dold = W[0]
update_ix = []
for ix,d in enumerate(W[1:]):
  if (np.array_equal(dold,d)==False):
    update_ix.append(ix)
    dold = d

update_ix.append(len(W) - 1) 
update_ix = np.asarray(update_ix)
update_ix = np.hstack([[0],update_ix])
print("update index in time indices")
print(update_ix)

# only go up to 20432

update_times = times_block1[keys[0]][0][update_ix]
print("")
print("update times in seconds")
print(update_times)

update_mins = update_times/60
print("")
print("update times in minutes")
print(update_mins)

tscale = update_ix[-1]/update_times[-1]
print("")
print("time scale conversion (index --> seconds): ", tscale)

n_upd = len(update_ix) - 2 # last update is only 337 so we want 17 updates

# save data_times
data_times = [update_ix, update_times]

# PATH = '/Users/mmadduri/OneDrive - UW/PhD_Research/Data/pickle-data-from-python/'
# with open(PATH + 'trial-related-data/times-decoder-update.pickle','wb') as handle:
#    pickle.dump(data_times,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(n_upd)

### STEP 4: Set up EMG data

In [ ]:
emgs = np.zeros((n_blocks, n_keys, n_conds, n_time, n_feat))

for iK, key in enumerate(keys):
    emgs[0, iK] = emgs_block1[key]
    emgs[1, iK] = emgs_block2[key]

### STEP 5: Set up the user input matrix (X)

User Input Matrix: FF + FB model

$$ EMG = W*u $$

$$ u = 
\begin{bmatrix}
r_x \\ r_y \\ \dot{r_x} \\ \dot{r_y}  \\ r_x - p_x \\ r_y - p_y \\ \dot{r_x} - \dot{p_x}   \\ \dot{r_y} - \dot{p_y} \\ 1
\end{bmatrix}
$$

where 
$$
FF = \begin{bmatrix}
r_x \\ r_y \\ \dot{r_x} \\ \dot{r_y} 
\end{bmatrix}
,
FB = \begin{bmatrix}
r_x - p_x \\ r_y - p_y \\ \dot{r_x} - \dot{p_x}   \\ \dot{r_y} - \dot{p_y}\end{bmatrix}
,
b = \begin{bmatrix}
 1
\end{bmatrix}
$$

So $W \in R^{64 \times 9}$

In [ ]:
poss_block1[keys[0]].shape

In [ ]:
x_dim = 8 # dimensions of the user input matrix 

# indices of the user input matrix by 
targ_pos_idx = [0, 1] # F0
targ_vel_idx = [2, 3] # F1
targ_pos_err_idx = [4, 5] # B0
targ_vel_err_idx = [6, 7] # B1
FF_idx = [0, 1, 2, 3]
FB_idx = [4, 5, 6, 7]

# this is the user input matrix
# will be set up as: n_time x x_dim
# this becomes: [targ_pos targ_vel targ_pos_err targ_vel_err]
# set to ones so that the last row is ones (for offset)
pos_vel_model = np.ones((n_blocks, n_keys, n_conds, n_time, x_dim + 1))

# set up saving cursor position
curs_pos = np.zeros((n_blocks, n_keys, n_conds, n_time, n_dim))

# set up saving target position
# already saved as ref_vels from previous code

# make sure that ref_vels is saved in the format we expect
assert(ref_vels.shape == (n_blocks, n_keys, n_conds, n_time, n_dim))
# make sure that format matches the other shapes
assert(dec_vels_block1[keys[0]].shape == ref_vels[0, 0].shape)

# set up saving decoded cursos velocity to be used to calculate velocity error
curs_vels = np.zeros((n_blocks, n_keys, n_conds, n_time, n_dim))

# set up saving target position error
targ_pos_err = np.zeros((n_blocks, n_keys, n_conds, n_time, n_dim))

# set up saving target velcoity error
targ_vels_err = np.zeros((n_blocks, n_keys, n_conds, n_time, n_dim))

## set up the inputs individually and then set up the matrix
# this needs to be done because the data is saved either in a dict and indexed by keys or in a matrix and indexed by iK
for iK, key in enumerate(keys):

    # save cursor position to calculate the error 
    curs_pos[0, iK] = poss_block1[key]
    curs_pos[1, iK] = poss_block2[key]

    # save the cursor velcity
    curs_vels[0, iK] = dec_vels_block1[key]
    curs_vels[1, iK] = dec_vels_block2[key]

    # targ pos error - this is currently the difference between the x and y position without absolute value
    targ_pos_err[0, iK] = (refs_block1[key] - poss_block1[key]) #/np.max((refs_block1[key] - poss_block1[key]), axis = 1)[:, np.newaxis] 
    targ_pos_err[1, iK] = (refs_block2[key] - poss_block2[key]) #/np.max((refs_block2[key] - poss_block2[key]), axis = 1)[:, np.newaxis]
    
    # targ velocity error - difference between target velocity and cursor velocity
    # same difference between x and y position without absolute value
    targ_vels_err[0, iK] = (ref_vels[0, iK] - dec_vels_block1[key]) #/np.max(int_vel_block1[key] - dec_vels_block1[key], axis = 1)[:, np.newaxis]
    targ_vels_err[1, iK]= (ref_vels[1, iK] - dec_vels_block2[key]) #/np.max(int_vel_block2[key] - dec_vels_block2[key], axis = 1)[:, np.newaxis]

### set up the pos_vel_model

# first 2 rows are target position
pos_vel_model[:, :, :, :, targ_pos_idx] = refs

# second 2 rows are target velocity
pos_vel_model[:, :, :, :, targ_vel_idx] = ref_vels

# next 2 rows are target - cursor position error
pos_vel_model[:, :, :, :, targ_pos_err_idx] = targ_pos_err

# last 2 rows are target - cursor velocity error
pos_vel_model[:, :, :, :, targ_vel_err_idx] = targ_vels_err

# make sure the last row is all 1's for offset
assert(pos_vel_model[:, :, :, :, -1].all() == 1)

### STEP 6: Calculate the User Encoders Using Linear Regression

In [ ]:
update_ix

In [ ]:
# get a lot of divide by 0 because so few points in r^2
# encoder estimation with intended velocity
# this takes about 20 seconds to run

encoder_matrix = np.zeros((n_blocks, n_keys, n_conds, n_upd, n_feat, x_dim+1)) # 2 x 14 x 8 x 64 x 5
r2_all_upd = np.zeros((n_blocks, n_keys, n_conds, n_upd))

# with the psuedoinverse
encoder_psi = np.zeros((n_blocks, n_keys, n_conds, n_upd, n_feat, x_dim+1)) # 2 x 14 x 8 x 64 x 5


x_data = pos_vel_model # target, target velocity 
y_data = emgs

for block in range(n_blocks):
    for key in range(n_keys):
        for cond in range(n_conds):
            for upd in range(n_upd):
                # if we print(upd), we want the last upd to be 16
                if upd > 16:
                    print("Check Indices of Encoder")
                # breaks down the code by decoder updates - so each encoder is estimated each time the decoder is updated
                # ends up being 17 updates since the last update is cut short in the experiment
                x_sample = x_data[block, key, cond, update_ix[upd]:update_ix[upd+1], 0:x_dim] # time x 8
                # check the shapes -- time x dim
                assert(x_sample.shape[0] >= update_ix[1]) 
                assert(x_sample.shape[-1] == x_dim) 
                y_sample = y_data[block, key, cond, update_ix[upd]:update_ix[upd+1]] # time x 64
                assert(y_sample.shape[0] >= update_ix[1]) 
                assert(y_sample.shape[-1] == n_feat) 
                weights, intercept, r2_avg = analysis.estimate_encoder_linear(x_sample, y_sample)
                
                assert(weights.shape == (n_feat, x_dim))
                # since the linear regression finds a y-intercept
                weights_affine = np.hstack((weights, intercept[:, np.newaxis]))
                assert(weights_affine.shape == (n_feat, x_dim + 1))

                encoder_matrix[block, key, cond, upd] = weights_affine
                r2_all_upd[block, key, cond, upd] = r2_avg

                # try with the psuedoinverse to verify that the method works
                # need to re-define x_sample here since we want the last rows of 1 in x_data for the affine term
                x_sample = x_data[block, key, cond, update_ix[upd]:update_ix[upd+1]]
                x_psi = np.linalg.pinv(x_sample.T)
                A_mtx = y_sample.T@x_psi
                assert(np.allclose(A_mtx, weights_affine))
                encoder_psi[block, key, cond, upd] = A_mtx

In [ ]:
# check that the linear regression and the psuedoinverse give the same answer 
# note: linear regression is much faster, might be better optimized
np.allclose(encoder_psi, encoder_matrix)

In [ ]:
# ## save the encoder model 
## commented out for codeocean/github

# idx_dict = {'FF_idx': FF_idx, 
#             'FB_idx': FB_idx,
#             'targ_pos_idx': targ_pos_idx, 
#             'targ_vel_idx': targ_vel_idx, 
#             'targ_pos_err_idx': targ_pos_err_idx,
#             'targ_vel_err_idx': targ_vel_err_idx}


# data = [encoder_matrix, r2_all_upd, idx_dict, pos_vel_model, curs_pos, curs_vels, Ws_block1, Ws_block2]

# PATH = '/Users/mmadduri/OneDrive - UW/PhD_Research/Data/pickle-data-from-python/encoder-estimation-data/'
# with open(PATH + 'encoder-estimation-model.pickle','wb') as handle:
#     pickle.dump(data,handle,protocol=pickle.HIGHEST_PROTOCOL)